In [2]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [3]:
test = open("data/test.txt","w",encoding='UTF-8')

p = requests.Session()                            # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup") # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
a=[]
for link in sel:
    a.append(link["href"])
    print('https://www.dcard.tw'+link.get('href'))

https://www.dcard.tw/f/makeup/p/235022153
https://www.dcard.tw/f/makeup/p/235021477
https://www.dcard.tw/f/makeup/p/235025010
https://www.dcard.tw/f/makeup/p/235022346
https://www.dcard.tw/f/makeup/p/235021551
https://www.dcard.tw/f/makeup/p/235026845
https://www.dcard.tw/f/makeup/p/235025318
https://www.dcard.tw/f/makeup/p/235021107
https://www.dcard.tw/f/makeup/p/235023535
https://www.dcard.tw/f/makeup/p/235029187
https://www.dcard.tw/f/makeup/p/235025794
https://www.dcard.tw/f/makeup/p/235026239
https://www.dcard.tw/f/makeup/p/235028318
https://www.dcard.tw/f/makeup/p/235024053
https://www.dcard.tw/f/makeup/p/235029094
https://www.dcard.tw/f/makeup/p/235023273
https://www.dcard.tw/f/makeup/p/235021138
https://www.dcard.tw/f/makeup/p/235027858
https://www.dcard.tw/f/makeup/p/235024620
https://www.dcard.tw/f/makeup/p/235024257
https://www.dcard.tw/f/makeup/p/235023687
https://www.dcard.tw/f/makeup/p/235025935
https://www.dcard.tw/f/makeup/p/235024483
https://www.dcard.tw/f/makeup/p/23

In [4]:
#a
url = "https://www.dcard.tw"+ a[1]
url

'https://www.dcard.tw/f/makeup/p/235021477'

根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [5]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)

,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235020702,發現冬天好適合用粉霜喔!,入冬後我臉已開始乾TT\n之前夏天用MUF粉餅 遮瑕力跟控油力都不錯\n但冬天用真的不夠保濕...,入冬後我臉已開始乾TT，之前夏天用MUF粉餅 遮瑕力跟控油力都不錯，但冬天用真的不夠保濕，最...,2020-12-21T02:26:52.040Z,2020-12-21T02:34:47.300Z,美妝,makeup,F,NaN


In [6]:
url = 'https://www.dcard.tw/_api/posts/235020702'  #測試1
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true'  #測試2
requ = requests.get(url)
rejs = requ.json()
#rejs

In [7]:
# slope the title
def Catch(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )

In [8]:
# 熱門美妝版測試
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)
print(df.shape)
df

(30, 10)


,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235022153,❄️聖誕妝容推薦❄️ 準備漂亮過節讓你 bling bling 惹人愛！,這週就是🎄聖誕節惹⛄️\n各位卡友們餐廳跟交換禮物都準備好了！\n現在應該是萬事俱備 只欠東...,這週就是聖誕節惹️，各位卡友們餐廳跟交換禮物都準備好了！，現在應該是萬事俱備 只欠東風，是聖...,2020-12-21T07:07:47.084Z,2020-12-21T10:13:21.728Z,美妝,makeup,D,NaN
1,235021477,🔥#圖多#試色 MAC唇膏眼影底妝失心瘋,哈摟大家好!🙋🏻\n今天是我隔離的最後一天了\n來檢討的是我黑五沒忍住爆買了MAC😂\n經典...,哈摟大家好！，今天是我隔離的最後一天了，來檢討的是我黑五沒忍住爆買了MAC，經典黑管唇膏打6...,2020-12-21T05:05:27.545Z,2020-12-21T06:22:01.027Z,美妝,makeup,F,NaN
2,235025010,#吐槽大會 那些我2021絕對不會購入的東西🌚,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下 \n所以...,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下，所以今天...,2020-12-21T14:45:24.926Z,2020-12-21T14:45:24.926Z,美妝,makeup,F,NaN
3,235022346,#分享 日常上學眼妝٩(˃̶͈̀௰˂̶͈́)و,大家午安\n今天想跟大家分享三個日常上學眼妝\n\n第一跟第二個都是用NYX的warm ne...,大家午安，今天想跟大家分享三個日常上學眼妝，第一跟第二個都是用NYX的warm neutra...,2020-12-21T07:37:10.545Z,2020-12-21T07:37:10.545Z,美妝,makeup,F,NaN
4,235021551,#分享 來一朵煙灰玫瑰，HERA #499,嗨嗨大家好～～\n先來一張人間香奈兒當鎮樓圖(⁎⁍̴̛ᴗ⁍̴̛⁎)\nhttps://i.i...,嗨嗨大家好～～先來一張人間香奈兒當鎮樓圖（⁎⁍̴̛ᴗ⁍̴̛⁎），是滴！今天要分享的是煎妮代言...,2020-12-21T05:16:38.008Z,2020-12-21T05:19:09.051Z,美妝,makeup,F,NaN
5,235026845,毛孔人必備的幾樣小可愛們✍,身為毛孔人，只要能讓毛孔縮小一咪咪\n都是讓人值得開心的事情阿～😙\n自己除了有固定使用針對...,身為毛孔人，只要能讓毛孔縮小一咪咪，都是讓人值得開心的事情阿～自己除了有固定使用針對毛孔的保...,2020-12-22T00:38:00.446Z,2020-12-22T00:38:00.446Z,美妝,makeup,F,NaN
6,235025318,舊款焦糖拿鐵的替代品,在康是美快打烊的時候 衝了一波他們的99元唇膏\n也不想造成店員困擾 於是也沒試色拿了一隻顏...,在康是美快打烊的時候 衝了一波他們的99元唇膏，也不想造成店員困擾 於是也沒試色拿了一隻顏色...,2020-12-21T15:29:36.622Z,2020-12-21T15:29:36.622Z,美妝,makeup,F,NaN
7,235021107,🔥3CE官網九宮格眼影盤買一送一 RIGHT NOW!!!,就是現在！！！\n官網聖誕優惠 11:00-15:00\n今天是九宮格眼影盤/霧面唇釉\n買...,就是現在！！！，官網聖誕優惠 11：00-15：00，今天是九宮格眼影盤/霧面唇釉，買一送一...,2020-12-21T03:57:57.576Z,2020-12-21T03:57:57.576Z,美妝,makeup,M,NaN
8,235023535,#分享 ［長文］盲買🇰🇷韓國5支唇膏來跟你們分享囉～,🦖平常在滑IG的時候，常常會看到各國唇膏的試色照片，也在收藏裡儲存了一大堆唇膏名單，所以這次...,🦖平常在滑IG的時候，常常會看到各國唇膏的試色照片，也在收藏裡儲存了一大堆唇膏名單，所以這次...,2020-12-21T10:56:19.212Z,2020-12-21T10:56:19.212Z,美妝,makeup,M,NaN
9,235029187,#分享 約會必備桃花眼妝💐-完美日記錦鯉盤,哈囉大家好這裡是最近很高產的兔子🐰\n之後可能就不會這麼常發文了\n想趕快把手邊能分享的產品...,哈囉大家好這裡是最近很高產的兔子，之後可能就不會這麼常發文了，想趕快把手邊能分享的產品分享出...,2020-12-22T08:35:30.926Z,2020-12-22T08:36:44.262Z,美妝,makeup,F,NaN


In [9]:
df.to_csv('data/Dcard_markeup.csv', index=False)

In [10]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(2):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
    print("the data is reading...",j)
print(df.shape)
df


the data is reading... 0
the data is reading... 1
(90, 10)


,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235022153,❄️聖誕妝容推薦❄️ 準備漂亮過節讓你 bling bling 惹人愛！,這週就是🎄聖誕節惹⛄️\n各位卡友們餐廳跟交換禮物都準備好了！\n現在應該是萬事俱備 只欠東...,這週就是聖誕節惹️，各位卡友們餐廳跟交換禮物都準備好了！，現在應該是萬事俱備 只欠東風，是聖...,2020-12-21T07:07:47.084Z,2020-12-21T10:13:21.728Z,美妝,makeup,D,NaN
1,235021477,🔥#圖多#試色 MAC唇膏眼影底妝失心瘋,哈摟大家好!🙋🏻\n今天是我隔離的最後一天了\n來檢討的是我黑五沒忍住爆買了MAC😂\n經典...,哈摟大家好！，今天是我隔離的最後一天了，來檢討的是我黑五沒忍住爆買了MAC，經典黑管唇膏打6...,2020-12-21T05:05:27.545Z,2020-12-21T06:22:01.027Z,美妝,makeup,F,NaN
2,235025010,#吐槽大會 那些我2021絕對不會購入的東西🌚,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下 \n所以...,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下，所以今天...,2020-12-21T14:45:24.926Z,2020-12-21T14:45:24.926Z,美妝,makeup,F,NaN
3,235022346,#分享 日常上學眼妝٩(˃̶͈̀௰˂̶͈́)و,大家午安\n今天想跟大家分享三個日常上學眼妝\n\n第一跟第二個都是用NYX的warm ne...,大家午安，今天想跟大家分享三個日常上學眼妝，第一跟第二個都是用NYX的warm neutra...,2020-12-21T07:37:10.545Z,2020-12-21T07:37:10.545Z,美妝,makeup,F,NaN
4,235021551,#分享 來一朵煙灰玫瑰，HERA #499,嗨嗨大家好～～\n先來一張人間香奈兒當鎮樓圖(⁎⁍̴̛ᴗ⁍̴̛⁎)\nhttps://i.i...,嗨嗨大家好～～先來一張人間香奈兒當鎮樓圖（⁎⁍̴̛ᴗ⁍̴̛⁎），是滴！今天要分享的是煎妮代言...,2020-12-21T05:16:38.008Z,2020-12-21T05:19:09.051Z,美妝,makeup,F,NaN
...,...,...,...,...,...,...,...,...,...,...
85,235024586,#請益 #請益 suqqu六色腮紅盤選擇,大家好～最近手癢想買東西，目前看上suqqu六色腮紅盤！真的好好看～～～\n不過由於膚色不算...,大家好～最近手癢想買東西，目前看上suqqu六色腮紅盤！真的好好看～～～不過由於膚色不算淺，...,2020-12-21T13:49:40.219Z,2020-12-21T13:49:40.219Z,美妝,makeup,F,NaN
86,235021357,"#問 京站,台北站前新光,遠百寶慶CHANEL",想請問這三間有沒有推薦的天使櫃哥或櫃姐？\n最近想買他們家的底妝產品～\n有爬文過但覺得評價...,想請問這三間有沒有推薦的天使櫃哥或櫃姐？最近想買他們家的底妝產品～有爬文過但覺得評價都很兩極...,2020-12-21T04:45:05.673Z,2020-12-21T04:52:55.916Z,美妝,makeup,F,NaN
87,235031304,#請益 年底各種優惠，大家都買了啥??,雙11一走又來個雙12\n雙11其實我很克制我自己\n\n因為我知道…\n12月才是真的可怕...,雙11一走又來個雙12，雙11其實我很克制我自己，因為我知道…，12月才是真的可怕~是個花錢...,2020-12-22T14:36:19.716Z,2020-12-22T14:36:19.716Z,美妝,makeup,F,NaN
88,235031109,Etude house眼影盤 改包裝？假貨？,大家先不要看到標題就急著罵\n冷靜一點( ´༎ຶㅂ༎ຶ`)\n\n我是在蝦皮上買的\n因為是...,大家先不要看到標題就急著罵，冷靜一點（ ´༎ຶㅂ༎ຶ`），我是在蝦皮上買的，因為是眼影新手，...,2020-12-22T14:10:07.442Z,2020-12-22T14:10:07.442Z,美妝,makeup,F,NaN


欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [ ]:
for k in range(0,10):
    post_data={
        "before":a[-1][9:18],
        "limit":"30",
        "popular":"true"
    }
    r = p.get("https://www.dcard.tw/_api/forums/makeup/posts",
              params=post_data, 
              headers = { "Referer": "https://www.dcard.tw/", 
                         "User-Agent": "Mozilla/5.0" })
    ##
    data2 = json.loads(r.text)
    for u in range(len(data2)):
        #Temporary_url = "/f/makeup/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
        Temporary_url = str(data2[u]["id"])
        a.append(Temporary_url)


In [ ]:
for k in range(0,10):
    post_data={
        "before":a[-1][9:18],
        "limit":"30",
        "popular":"true"
    }
    r = p.get("https://www.dcard.tw/_api/forums/makeup/posts",
              params=post_data, 
              headers = { "Referer": "https://www.dcard.tw/", 
                         "User-Agent": "Mozilla/5.0" })
    ##
    data2 = json.loads(r.text)
data2

In [ ]:
for k in range(0,10):
    post_data={
        "before":a[-1][9:18],
        "limit":"30",
        "popular":"true"
    }
    r = p.get("https://www.dcard.tw/_api/forums/pet/posts",
              params=post_data, 
              headers = { "Referer": "https://www.dcard.tw/", "User-Agent": "Mozilla/5.0" })
    data2 = json.loads(r.text)
    for u in range(len(data2)):
        Temporary_url = "/f/pet/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
        a.append(Temporary_url)
j=0 #為了印頁數
q=0 #為了印張數

In [ ]:
# for k in range(0,10):
#     post_data={
#         "before":a[-1][9:18],
#         "limit":"30",
#         "popular":"true"
#     }
#     r = p.get("https://www.dcard.tw/_api/forums/pet/posts",
#               params=post_data, headers = { "Referer": "https://www.dcard.tw/", "User-Agent": "Mozilla/5.0" })

發現GET來的檔案格式是JSON，那小編比較不會處理JSON怎辦?可以把他轉成Python就好啦，並組合好網址放進list

In [ ]:
data2 = json.loads(r.text)
for u in range(len(data2)):
    Temporary_url = "/f/pet/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
    a.append(Temporary_url)

接下來就是將list裡的URL，GET網頁出來

In [ ]:
j=0
q=0
for i in a[2:]:
    url = "https://www.dcard.tw"+i
    j+=1
    print ("第",j,"頁的URL為:"+url)
    #file.write("temperature is {} wet is {}%\n".format(temperature, humidity))
    test.write("第 {} 頁的URL為: {} \n".format(j,url))
    url=requests.get(url)
    soup = BeautifulSoup(url.text,"html.parser")

運用BeautifulSoup查看是否有符合圖片的標籤，然後運用上次所學，將圖片存檔

In [ ]:
sel_jpg = soup.select("div.Post_content_NKEl9 div div div img.GalleryImage_image_3lGzO")
for c in sel_jpg:
    q+=1
    print("第",q,"張:",c["src"])
    test.write("%\n""第 {} 張: {} \n".format(q,c["src"])) 
    pic=requests.get(c["src"])
    img2 = pic.content
    pic_out = open("spider/pet/"+str(q)+".png",'wb')
    pic_out.write(img2)
    pic_out.close()

In [ ]:
test.close()
print("爬蟲結束")

完整的程式碼(可改range將文章變多)

In [ ]:
#藉由首頁取得所有文章的URL
import requests
from bs4 import BeautifulSoup
import json

test = open("data/test.txt","w",encoding='UTF-8')


p = requests.Session()
url=requests.get("https://www.dcard.tw/f/pet")
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("div.PostList_wrapper_2BLUM a.PostEntry_root_V6g0r")
a=[]
for s in sel:
    a.append(s["href"])
url = "https://www.dcard.tw"+ a[2]

for k in range(0,10):
        post_data={
            "before":a[-1][9:18],
            "limit":"30",
            "popular":"true"
        }
        r = p.get("https://www.dcard.tw/_api/forums/pet/posts",params=post_data, headers = { "Referer": "https://www.dcard.tw/", "User-Agent": "Mozilla/5.0" })
        data2 = json.loads(r.text)
        for u in range(len(data2)):
            Temporary_url = "/f/pet/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
            a.append(Temporary_url)
j=0 #為了印頁數
q=0 #為了印張數
for i in a[2:]:
    url = "https://www.dcard.tw"+i
    j+=1
    print ("第",j,"頁的URL為:"+url)
    #file.write("temperature is {} wet is {}%\n".format(temperature, humidity))
    test.write("第 {} 頁的URL為: {} \n".format(j,url))
    url=requests.get(url)
    soup = BeautifulSoup(url.text,"html.parser")
    sel_jpg = soup.select("div.Post_content_NKEl9 div div div img.GalleryImage_image_3lGzO")
    for c in sel_jpg:
        q+=1
        print("第",q,"張:",c["src"])
        test.write("%\n""第 {} 張: {} \n".format(q,c["src"])) 
        pic=requests.get(c["src"])
        img2 = pic.content
        pic_out = open("spider/pet/"+str(q)+".png",'wb')
        pic_out.write(img2)
        pic_out.close()

test.close()
print("爬蟲結束")